In [1]:
from models.sos import ScalarOnScalarModel
from optimizers.nbdo import NBDO
from bases.bspline import BSplineBasis
from models.sof import ScalarOnFunctionModel
import numpy as np

In [2]:
# SoS
model = ScalarOnScalarModel(Kx=3, criterion="A", order=2)
nbdo = NBDO(model=model, latent_dim=4, seed=42)
nbdo.compute_train_set(num_designs=1_000, runs=25)
nbdo.fit(epochs=1_000, patience=100, batch_size=256)
crit, design = nbdo.optimize(n_calls=30)
np.round(crit,2)

np.float64(1.24)

In [3]:
np.round(design)

array([[ 1., -1.,  1.],
       [-1., -1., -1.],
       [-1.,  0.,  0.],
       [ 0., -1., -0.],
       [-1., -0., -1.],
       [ 1., -1., -1.],
       [ 0., -0., -0.],
       [ 1.,  1., -1.],
       [ 1.,  1.,  0.],
       [-0., -1.,  0.],
       [ 0., -0., -1.],
       [ 0.,  1.,  0.],
       [ 0.,  0., -0.],
       [-1.,  1.,  1.],
       [-1., -1.,  1.],
       [ 1., -0.,  1.],
       [ 0.,  1.,  1.],
       [-0.,  0.,  1.],
       [-0., -0.,  1.],
       [ 1.,  1.,  1.],
       [-1.,  1., -1.],
       [-0.,  1., -1.],
       [-1.,  0.,  0.],
       [ 1., -0., -0.],
       [-1.,  1.,  0.]], dtype=float32)

In [4]:
from diagnostics import (
    info_matrix, eigen_spectrum, condition_number,
    leverage_diag, prediction_variance,
)

M = info_matrix(model, design)
eigvals, _ = eigen_spectrum(M)
kappa = condition_number(M)
print(f"p = {M.shape[0]}, λ_min = {eigvals[-1]:.3e}, λ_max = {eigvals[0]:.3e}, κ2 = {kappa:.2e}")

h = leverage_diag(model, design)
print(f"leverage stats: min={h.min():.3f}, max={h.max():.3f}, mean={h.mean():.3f}")


p = 10, λ_min = 3.458e+00, λ_max = 5.728e+01, κ2 = 1.66e+01
leverage stats: min=0.179, max=0.748, mean=0.400


In [5]:
# # SoF
# xB, bB = BSplineBasis(0,4), BSplineBasis(0,2)
# nbdo2 = NBDO(model=ScalarOnFunctionModel([(xB,bB)], criterion="D", intercept=True), latent_dim=2, seed=42, verbose=False)
# nbdo2.compute_train_set(num_designs=1_000, runs=12)
# nbdo2.fit(epochs=10, patience=5, batch_size=256)
# crit2, design2 = nbdo2.optimize(n_calls=5)
# crit2

In [6]:
# # examples/sof_two_factor_smoke.py
# import numpy as np
# from bases.bspline import BSplineBasis
# from models.sof import ScalarOnFunctionModel
# from optimizers.nbdo import NBDO

# # Factor 1
# x1 = BSplineBasis(degree=0, total_knots_num=4)  # Kx1 = x1.num_basis()
# b1 = BSplineBasis(degree=0, total_knots_num=3)  # Kb1 = b1.num_basis()

# # Factor 2
# x2 = BSplineBasis(degree=1, total_knots_num=5)  # different degree/knots ok
# b2 = BSplineBasis(degree=0, total_knots_num=4)

# model = ScalarOnFunctionModel(
#     basis_pairs=[(x1, b1), (x2, b2)],
#     criterion="A",
#     intercept=True
# )

# # End-to-end smoke
# opt = NBDO(model=model, latent_dim=2, seed=0)
# opt.compute_train_set(num_designs=64, runs=25)
# opt.fit(epochs=5, batch_size=32, patience=3)     # short & cheap
# best_cr, best_des = opt.optimize(n_calls=8)      # short BO
# print("criterion:", np.round(best_cr,2)) # 10.93
# print("design shape:", best_des.shape)  # expected (N, model.Kx)
